# Final Project Notebook

**University of California, Berkeley**

**MIDS Program** - W207 Applied Machine Learning

**Section 7** - Mackenzie Austin, Emanuel Mejía, Ibrahim Shareef

In [ ]:
# ! pip install imblearn
# ! pip install graphviz

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import math
import warnings
warnings.filterwarnings('ignore')

#from PIL import Image
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
from matplotlib import pyplot as plt
import tensorflow as tf
#import sklearn
from tensorflow import keras
from keras import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#tf.get_logger().setLevel('INFO')
import imblearn
from imblearn.over_sampling import SMOTE
from timeit import default_timer as timer

# import sklearn
from sklearn import tree
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

start = timer()

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Download and Load Data </div>

In [ ]:
#setup dataset

# !mkdir input
# !cd input && kaggle datasets download -d sachinkumar413/alzheimer-mri-dataset && tar -xf alzheimer-mri-dataset.zip

In [ ]:
X = np.empty((6400, 128, 128, 3))
Y = np.empty(6400, dtype=int)
label_names = ['Non Demented', 'Very Mild Demented', 'Mild Demented', 'Moderate Demented']
index = 0
for subdir, dirs, files in os.walk('./input/Dataset/Non_Demented'):
    
    for file in files:
        img = load_img(os.path.join(subdir, file), target_size=(128,128))
        #print(type(img))
        #print(img.format)
        #print(img.mode)
        #print(img.size)
        
#         for grayscale
#         img_arr = img_to_array(img.convert('L'))
#         print(img_arr.shape)
#         print(img_arr)
#         img_arr = img_arr.reshape((128,-1))
#         print(img_arr.shape)
#         print(img_arr)
        
        img_arr = img_to_array(img)
        X[index] = img_arr
        Y[index] = 0
        index += 1
        
for subdir, dirs, files in os.walk('./input/Dataset/Very_Mild_Demented'):
    for file in files:
        img = load_img(os.path.join(subdir, file), target_size=(128,128))
        img_arr = img_to_array(img)
        X[index] = img_arr
        Y[index] = 1
        index += 1
        
for subdir, dirs, files in os.walk('./input/Dataset/Mild_Demented'):
    for file in files:
        img = load_img(os.path.join(subdir, file), target_size=(128,128))
        img_arr = img_to_array(img)
        X[index] = img_arr
        Y[index] = 2
        index += 1
        
for subdir, dirs, files in os.walk('./input/Dataset/Moderate_Demented'):
    for file in files:
        img = load_img(os.path.join(subdir, file), target_size=(128,128))
        img_arr = img_to_array(img)
        X[index] = img_arr
        Y[index] = 3
        index += 1
        
print(X.shape)
print(Y.shape)

In [ ]:
diagnoses = pd.Series(Y)
diagnoses = diagnoses.replace([0, 1, 2, 3], label_names)
diag_df = pd.DataFrame({'count': diagnoses.value_counts()})
diag_df['%'] = round(diag_df['count'] / len(Y)*100,0)

ax = plt.axes()
plt.bar(diag_df.index, diag_df['count'], align = 'center')
ax.set_xticks([0,1,2,3,4])

# Spine formatting
ax.set_facecolor(color='white')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('cornflowerblue')
ax.spines['bottom'].set_color('cornflowerblue')

# Tick formatting
ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
ax.tick_params(axis='x', which='both', color='cornflowerblue', 
               colors = 'gray', rotation = 30)

# Grid formatting
ax.grid(False)
vals = ax.get_yticks()
for tick in vals:
    ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='cornflowerblue', zorder=1)

# Setting text labels according to the count of df
rects = ax.patches
percentages = diag_df['%']
counts = diag_df['count']

for rect, label, cnt in zip(rects, percentages, counts):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height + 0.01, str(label) + '%',
            ha='center', va='bottom', color = 'gray',weight='normal')
    height = rect.get_height() + 200
    ax.text(rect.get_x() + rect.get_width() / 2, height + 0.01, cnt,
            ha='center', va='bottom', color = 'royalblue',weight='bold')
    
# Set title
plt.suptitle("Classification", x = 0.123, y = 1.06, ha = 'left', weight='bold', color = 'royalblue', 
             size=20)

# Set title
ax.set_title("Initial Distribution", weight='normal', color = 'dimgray', 
             style = 'italic', pad=14, loc='left', size=15)

# Set x-axis label
ax.set_xlabel("Class", labelpad=20, weight='bold', size=10, color='gray')

# Set y-axis label
ax.set_ylabel("Counts", labelpad=20, weight='bold', size=10, color='gray')

diag_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Class Imbalance Sections </div>

In [ ]:
# seed randoms in one place

random.seed(5678)
np.random.seed(5678)
tf.random.set_seed(5678)

In [ ]:
##Fix Data Imbalance using SMOTE

orig_shape = X.shape
X = np.reshape(X, (orig_shape[0], orig_shape[1] * orig_shape[2] * orig_shape[3]))
oversample = SMOTE()
print('X before smote: ', X.shape)
print('Y before smote: ', Y.shape)
X, Y = oversample.fit_resample(X, Y)
X = np.reshape(X, (X.shape[0], orig_shape[1], orig_shape[2], orig_shape[3]))
print('X after smote: ', X.shape)
print('Y after smote: ', Y.shape)

In [ ]:
diagnoses = pd.Series(Y)
diagnoses = diagnoses.replace([0, 1, 2, 3], label_names)
diag_df = pd.DataFrame({'count': diagnoses.value_counts()})
diag_df['%'] = round(diag_df['count'] / len(Y)*100,0)
diag_df.index.str.strip()
diag_df = diag_df.reindex(label_names)

ax = plt.axes()
plt.bar(diag_df.index, diag_df['count'], align = 'center')
ax.set_xticks([0,1,2,3,4])

# Spine formatting
ax.set_facecolor(color='white')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('cornflowerblue')
ax.spines['bottom'].set_color('cornflowerblue')

# Tick formatting
ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
ax.tick_params(axis='x', which='both', color='cornflowerblue', 
               colors = 'gray', rotation = 30)

# Grid formatting
ax.grid(False)
vals = ax.get_yticks()
for tick in vals:
    ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='cornflowerblue', zorder=1)

# Setting text labels according to the count of df
rects = ax.patches
percentages = diag_df['%']
counts = diag_df['count']

for rect, label, cnt in zip(rects, percentages, counts):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height + 0.01, str(label) + '%',
            ha='center', va='bottom', color = 'gray',weight='normal')
    height = rect.get_height() + 200
    ax.text(rect.get_x() + rect.get_width() / 2, height + 0.01, cnt,
            ha='center', va='bottom', color = 'royalblue',weight='bold')
    
# Set title
plt.suptitle("Classification", x = 0.123, y = 1.06, ha = 'left', weight='bold', color = 'royalblue', 
             size=20)

# Set title
ax.set_title("After Imbalance Treatment", weight='normal', color = 'dimgray', 
             style = 'italic', pad=14, loc='left', size=15)

# Set x-axis label
ax.set_xlabel("Class", labelpad=20, weight='bold', size=10, color='gray')

# Set y-axis label
ax.set_ylabel("Counts", labelpad=20, weight='bold', size=10, color='gray')

diag_df

In [ ]:
#shuffle input
#np.random.seed(0)
indices = np.arange(X.shape[0])
shuffled_indices = np.random.permutation(indices)
X = X[shuffled_indices]
Y = Y[shuffled_indices]

# Use a ~80/20 train/test split.
idx = (int)(X.shape[0] * 0.8)
X_train = X[:idx]
Y_train = Y[:idx]
X_test = X[idx:]
Y_test = Y[idx:]

X_train = X_train / 255
X_test = X_test / 255

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">PCA</div>

In [ ]:
X_train_pca = X_train.reshape(X_train.shape[0], (X_train.shape[1] * X_train.shape[2] * X_train.shape[3]))
X_test_pca = X_test.reshape(X_test.shape[0], (X_test.shape[1] * X_test.shape[2] * X_test.shape[3]))

# Creating a principal component analysis model
# Enough components to keep around 85% of the variance in the original data 
pca = PCA(n_components = 225)

# Feeding the independent variables to the PCA model
X_train_pca = pca.fit_transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)

print(X_train_pca.shape)
print(X_test_pca.shape)

In [ ]:
pca.n_components

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">End Class Imbalance Section</div>

In [ ]:
# Showing 5 examples of each class

classes = 4

examples = 5

pic_counter = [0] * classes
i = 0

# Create a figure with subplots.
# As a grid of # product_classes x # examples_per_each

fig, axs = plt.subplots(nrows = classes, ncols= examples, figsize=(10,classes * 2))

# Keep searching until we find all we need

while sum(pic_counter) < (classes * examples):
    
    image = X[i]
    label = int(Y[i])
    label_name = label_names[label]
    
    # Add a picture to the grid 
    # if we haven't find all the examples we need
    # for that product class
    if pic_counter[label] < examples:
        axs[label][pic_counter[label]].imshow(image.astype("uint8"))
        axs[label][pic_counter[label]].set_title(label_name)
        axs[label][pic_counter[label]].axis('off')
        pic_counter[label] += 1
    
    i += 1

plt.show()

In [ ]:
def compute_model_performance_metrics(model, x_test, y_test):
    #Record other performance metrics
    x_test_predict = model.predict(x_test, verbose=0)
    x_predict_classes =np.argmax(x_test_predict, axis=1)
    
    accuracy = accuracy_score(y_test, x_predict_classes)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, x_predict_classes, average='macro')
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, x_predict_classes, average='macro')
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, x_predict_classes, average='macro')
    #auc = roc_auc_score(Y_test, yhat_probs, multi_class='ovr')
    
    metrics = {
        "accuracy": format(accuracy, '.4f'),
        "precision": format(precision, '.4f'),
        "recall": format(recall, '.4f'),
        "f1_score": format(f1, '.4f')
    }
    
    return metrics

In [ ]:
def compute_model_performance_metrics_cnn_lstm(model, x_test, y_test):
    #Record other performance metrics
    x_test_predict = model.predict(x_test, verbose=0)

    x_test_predict_shape = x_test_predict.shape
    x_test_predict = np.reshape(x_test_predict, (x_test_predict_shape[0] * x_test_predict_shape[1], x_test_predict_shape[2]))

    y_test_seq_flat = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1]))

    x_predict_classes = np.argmax(x_test_predict, axis=1)
    
    accuracy = accuracy_score(y_test_seq_flat, x_predict_classes)
    # precision tp / (tp + fp)
    precision = precision_score(y_test_seq_flat, x_predict_classes, average='macro')
    # recall: tp / (tp + fn)
    recall = recall_score(y_test_seq_flat, x_predict_classes, average='macro')
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test_seq_flat, x_predict_classes, average='macro')
    
    metrics = {
        "accuracy": format(accuracy, '.4f'),
        "precision": format(precision, '.4f'),
        "recall": format(recall, '.4f'),
        "f1_score": format(f1, '.4f')
    }
    
    return metrics

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Multi-Class Logistic Regression </div>

In [ ]:
def build_logreg_model(n_classes = 4, 
                       learning_rate = 0.01, 
                       activation = 'softmax'):
    """Build a multi-class logistic regression model using Keras.

    Args:
        n_classes: Number of classes in the dataset
        learning_rate: The desired learning rate for SGD.

    Returns:
        model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    
    print('Activation function: ' + activation)
    print('Optimizer: SGD')
    print('Learning Rate: ' + str(learning_rate))
    
    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
      units = n_classes,
      activation = activation
    ))

    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)

    model.compile(loss = 'sparse_categorical_crossentropy', 
                  optimizer = optimizer, 
                  metrics = ['accuracy'])
    return model

In [ ]:
def train_and_evaluate_logreg(x_train,
                              y_train,
                              x_test,
                              y_test,
                              activation = 'softmax',
                              optimizer = 'SGD',
                              learning_rate = 0.001,
                              num_epochs = 20, 
                              verb = False):
    
    print('Number of Epochs: ' + str(num_epochs))
    
    model = build_logreg_model(n_classes = len(label_names), 
                               learning_rate = learning_rate,
                               activation = activation)

    history = model.fit(
        x = x_train,
        y = y_train,
        epochs = num_epochs,
        batch_size=64,
        verbose = verb,
        validation_split=0.1)

    hist = history.history
    train_accuracy = hist['accuracy']
    val_accuracy = hist['val_accuracy']
    num_params = model.count_params()
    test_predictions = np.argmax(model.predict(x_test, verbose = verb), axis=-1)
    test_accuracy = model.evaluate(x = x_test, y = y_test, verbose = verb,
                               return_dict=True)['accuracy']
  
    # Create a confusion matrix as a 2D array.
    confusion_matrix = tf.math.confusion_matrix(y_test, test_predictions)
    
    ## Plot mean CV accuracies for Tree Depth
    fig = plt.figure(figsize=(15, 5))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(train_accuracy, label = "Training Accuracy", 
         color = 'royalblue', marker = '.', lw = 1)
    ax.plot(val_accuracy, label = "Validation Accuracy", 
             color = 'mediumblue', marker = 'D', lw = 2)
    
    # Spine formatting
    ax.set_facecolor(color='white')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_color('cornflowerblue')
    ax.spines['bottom'].set_color('cornflowerblue')
    
    # Tick formatting
    ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
    ax.tick_params(axis='x', which='both', colors = 'gray')
    ax.set_xticks(range(num_epochs + 1))
    
    # Grid formatting
    ax.grid(False)
    vals = ax.get_yticks()
    for tick in vals:
        ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)
    
    # Set title
    plt.suptitle("Accuracy vs. Number of Epochs", x = 0.123, y = 1, 
                 ha = 'left', weight='bold', color = 'mediumblue', 
                 size = 18)

    # Set title
    ax.set_title("Training and Validation", weight='normal', color = 'dimgray', 
                 style = 'italic', pad=10, loc='left', size=15)
    
    ax.legend(loc = 4, fontsize = 'medium', 
              shadow = True,
              edgecolor = 'blue',
              labelcolor = ['royalblue','mediumblue'], 
              facecolor = 'ivory')
    
    ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
    ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
    ax.set_ylim(top = 1.01)

    # Use a heatmap plot to display Confusion Matrix.
    ax = fig.add_subplot(1, 2, 2)
    ax = sns.heatmap(confusion_matrix, annot = True, fmt = '.3g', cmap = 'Blues',
                     xticklabels = label_names, yticklabels = label_names, cbar = False)
    
    # Add axis labels.
    ax.tick_params(axis='x', which='both', length=0, colors = 'gray', rotation = 90)
    ax.set_xlabel('Predicted Label', color = 'gray', weight = 'bold')
    ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
    ax.set_ylabel('True Label', color = 'gray', weight = 'bold')

    ax.set_title('CONFUSION MATRIX', 
                 weight='bold', color = 'mediumblue', pad=14, loc='left', size=18)

    ax.text(x = 4, y = -0.22, ha = 'right',
            s = 'TEST ACC: ' + str(round(test_accuracy*100,2)) + '%',
            color = 'mediumblue', weight='bold')
    
    plt.subplots_adjust(wspace=0.5)
    plt.show()
    
    return [test_accuracy, num_params]

In [ ]:
exp_ind = ['Experiment_' + str(i) for i in range(1,5)]
acts = (['sigmoid'] * 2 + ['softmax'] * 2)
opts = (['SGD'] * 4)
rates = ([0.01] + [0.001]) * 2
epchs = [20] * 4
params = [0] * 4
accs = [0] * 4

experiments_lrdf = pd.DataFrame({'ACTIVATION': acts, 
                                 'OPTIMIZER': opts, 
                                 'LEARNING RATE': rates,
                                 'EPOCHS': epchs, 
                                 '# PARAMETERS': params,
                                 'TEST ACCURACY': accs,
                                 '# PARAMETERS (PCA)': params,
                                 'TEST ACCURACY (PCA)': accs}, 
                                index = exp_ind)

In [ ]:
for i in range (1,len(experiments_lrdf.index)+1):
    print('Experiment_' + str(i) + '\n')
    accu, param = train_and_evaluate_logreg(X_train, Y_train, X_test, Y_test,
                                            activation = experiments_lrdf.loc['Experiment_' + str(i), 'ACTIVATION'], 
                                            optimizer = experiments_lrdf.loc['Experiment_' + str(i), 'OPTIMIZER'], 
                                            learning_rate = experiments_lrdf.loc['Experiment_' + str(i), 'LEARNING RATE'])

    experiments_lrdf.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_lrdf.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    
    print('Experiment_' + str(i) + '_PCA\n')
    accu_pca, param_pca = train_and_evaluate_logreg(X_train_pca, Y_train, X_test_pca, Y_test,
                                            activation = experiments_lrdf.loc['Experiment_' + str(i), 'ACTIVATION'], 
                                            optimizer = experiments_lrdf.loc['Experiment_' + str(i), 'OPTIMIZER'], 
                                            learning_rate = experiments_lrdf.loc['Experiment_' + str(i), 'LEARNING RATE'])

    experiments_lrdf.loc['Experiment_' + str(i), '# PARAMETERS (PCA)'] = param_pca
    experiments_lrdf.loc['Experiment_' + str(i), 'TEST ACCURACY (PCA)'] = accu_pca

In [ ]:
experiments_lrdf

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">K-Means Clustering</div>

In [ ]:
def kmean_models(init_clusters,  
                 x_train, 
                 y_train,
                 x_test,
                 y_test):
    
    totl_rows = 2
    totl_cols = 5
    
    exp_ind = ['Experiment_' + str(i) for i in range(1,(totl_rows * totl_cols)+1)]
    kmean_df = pd.DataFrame(index = exp_ind)

    fig, axs = plt.subplots(nrows = totl_rows, ncols = totl_cols, figsize = (20, 8))

    for i in range(0, totl_rows * totl_cols):
        row = math.trunc(i/totl_cols)
        col = i%totl_cols

        num_clusters = init_clusters ** (i)

        kmean = KMeans(n_clusters = num_clusters, 
                       init = 'k-means++',
                       random_state = 0)

        kmean.fit(X_train_pca, Y_train)

        kmean_train = kmean.predict(X_train_pca)

        kmean_translator = pd.DataFrame({'CLUSTER': kmean_train, 
                                         'LABEL': Y_train})

        translator = pd.Series.tolist(kmean_translator.groupby(['CLUSTER']).agg(pd.Series.mode)['LABEL'])
        
        kmean_train_pred = kmean.predict(X_train_pca)
        kmean_train_pred = pd.Series(kmean_train_pred)
        kmean_train_pred = kmean_train_pred.replace(range(0, num_clusters), translator)
        train_acc = sum(kmean_train_pred == Y_train)/len(Y_train)
        
        kmean_predictions = kmean.predict(X_test_pca)
        kmean_predictions = pd.Series(kmean_predictions)
        kmean_predictions = kmean_predictions.replace(range(0, num_clusters), translator)
        test_acc = sum(kmean_predictions == Y_test)/len(Y_test)

        kmean_confusion_matrix = tf.math.confusion_matrix(Y_test, kmean_predictions)

        sns.heatmap(ax = axs[row][col], data = kmean_confusion_matrix, annot=True, fmt='.3g', cmap='Blues',
                        xticklabels = (label_names if row + 1 == totl_rows else 'auto'), 
                        yticklabels = (label_names if col == 0 else 'auto'), cbar=False)

        # Add axis labels.
        if row + 1 == totl_rows:
            axs[row][col].set_xlabel('Predicted Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='x', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_xticks([])

        if col == 0:
            axs[row][col].set_ylabel('True Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='y', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_yticks([])

        axs[row][col].set_title("# CLUSTERS: " + str(num_clusters) + '    TEST ACC: ' + str(round(test_acc*100,2)) + '%', 
                                    weight='bold', color = 'mediumblue', pad=14, loc='center', size=12)
        
        kmean_df.loc['Experiment_' + str(i + 1), '# CLUSTERS'] = num_clusters
        kmean_df.loc['Experiment_' + str(i + 1), 'TRAIN ACCURACY'] = train_acc
        kmean_df.loc['Experiment_' + str(i + 1), 'TEST ACCURACY'] = test_acc

    plt.show()

    return kmean_df

## Plot accuracies vs. k values
experiments_kmean_df = kmean_models(2, X_train_pca, Y_train, X_test_pca, Y_test)

experiments_kmean_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Nearest Neighbors </div>

In [ ]:
## Create helper function for plotting the training accuracy vs. 
## cross-validation accuracy plot with various depths
def knn_acc_plot(start: int, 
                 end: int, 
                 x_train, 
                 y_train,
                 x_test,
                 y_test):
    ## Set k range and initialize df
    k_range = range(start, end)
    exp_ind = ['Experiment_' + str(i) for i in range(1,end-start+1)]
    knn_df = pd.DataFrame(index = exp_ind)
    totl_cols = 4
    totl_rows = math.ceil((end-start-1)/totl_cols)
    
    fig, axs = plt.subplots(nrows = totl_rows, ncols= totl_cols, figsize = (5 * totl_cols, 4 * totl_rows))
    i = 0
    
    for k in k_range:
        row = math.trunc(i/totl_cols)
        col = i%totl_cols
        knn = KNeighborsClassifier(n_neighbors = k)
        knn.fit(x_train, y_train)
        accuracy = knn.score(x_train, y_train)
        scores = cross_val_score(knn, x_train, y_train, cv=5, scoring='accuracy')
        test_accuracy = knn.score(x_test, y_test)
        knn_predictions = knn.predict(x_test)
        knn_confusion_matrix = tf.math.confusion_matrix(y_test, knn_predictions)
        
        sns.heatmap(ax = axs[row][col], data = knn_confusion_matrix, annot=True, fmt='.3g', cmap='Blues',
                    xticklabels = (label_names if row + 1 == totl_rows else 'auto'), 
                    yticklabels = (label_names if col == 0 else 'auto'), cbar=False)

        # Add axis labels.
        if row + 1 == totl_rows:
            axs[row][col].set_xlabel('Predicted Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='x', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_xticks([])
            
        if col == 0:
            axs[row][col].set_ylabel('True Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='y', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_yticks([])
        
        axs[row][col].set_title("NEIGHBORS (k): " + str(k) + '    TEST ACC: ' + str(round(test_accuracy*100,2)) + '%', 
                                weight='bold', color = 'mediumblue', pad=14, loc='center', size=12)
        
        knn_df.loc['Experiment_' + str(i + 1), 'NEIGHBORS (k)'] = k
        knn_df.loc['Experiment_' + str(i + 1), 'CV ACCURACY'] = scores.mean()
        knn_df.loc['Experiment_' + str(i + 1), 'TRAIN ACCURACY'] = accuracy.mean()
        knn_df.loc['Experiment_' + str(i + 1), 'TEST ACCURACY'] = test_accuracy
        
        i += 1
    plt.show()

    ## Plot mean CV accuracies for k
    ax = plt.axes()
    plt.plot(k_range, knn_df['TRAIN ACCURACY'], label = "Training Accuracy", 
         color = 'royalblue', marker = '.', lw = 1)
    plt.plot(k_range, knn_df['CV ACCURACY'], label = "Cross-Val Accuracy", 
             color = 'mediumblue', marker = 'D', lw = 2)
    
    # Spine formatting
    ax.set_facecolor(color='white')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_color('cornflowerblue')
    ax.spines['bottom'].set_color('cornflowerblue')
    
    # Tick formatting
    ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
    ax.tick_params(axis='x', which='both', colors = 'gray')
    ax.set_xticks(k_range)
    
    # Grid formatting
    ax.grid(False)
    vals = ax.get_yticks()
    for tick in vals:
        ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)
    
    # Set title
    plt.suptitle("Mean Accuracies vs. k after Initial Split", x = 0.123, y = 1.06, 
                 ha = 'left', weight='bold', color = 'mediumblue', 
                 size = 18)

    # Set title
    ax.set_title("Training and Cross-Validation", weight='normal', color = 'dimgray', 
                 style = 'italic', pad=14, loc='left', size=15)
    
    ax.legend(loc = 3, fontsize = 'medium', 
              shadow = True,
              edgecolor = 'blue',
              labelcolor = ['royalblue','mediumblue'], 
              facecolor = 'ivory')
    
    ax.set_xlabel('Neighbors (k)', color = 'gray', weight = 'bold')
    ax.set_ylabel('Mean Accuracy', color = 'gray', weight = 'bold')
    ax.set_ylim(top = 1.01)
    plt.show()
    
    return knn_df

## Plot accuracies vs. k values
experiments_knn_df = knn_acc_plot(1, 21, X_train_pca, Y_train, X_test_pca, Y_test)

experiments_knn_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Decision Trees</div>

In [ ]:
## Create helper function for plotting the training accuracy vs. 
## cross-validation accuracy plot with various depths
def dt_acc_plot(start: int, 
                end: int, 
                x_train, 
                y_train,
                x_test,
                y_test):
    ## Set depth range and initialize lists
    t_depth = range(start, end)
    exp_ind = ['Experiment_' + str(i) for i in range(1,end-start+1)]
    tree_df = pd.DataFrame(index = exp_ind)
    totl_cols = 4
    totl_rows = math.ceil((end-start-1)/totl_cols)
    
    fig, axs = plt.subplots(nrows = totl_rows, ncols= totl_cols, figsize = (5 * totl_cols, 4 * totl_rows))
    i = 0
    
    for lvl in t_depth:
        row = math.trunc(i/totl_cols)
        col = i%totl_cols
        dtree = tree.DecisionTreeClassifier(random_state = 0, max_depth = lvl)
        dtree.fit(x_train, y_train)
        accuracy = dtree.score(x_train, y_train)
        scores = cross_val_score(dtree, x_train, y_train, cv=5, scoring='accuracy')
        test_accuracy = dtree.score(x_test, y_test)
        tree_predictions = dtree.predict(x_test)
        tree_confusion_matrix = tf.math.confusion_matrix(y_test, tree_predictions)
        
        sns.heatmap(ax = axs[row][col], data = tree_confusion_matrix, annot=True, fmt='.3g', cmap='Blues',
                    xticklabels = (label_names if row + 1 == totl_rows else 'auto'), 
                    yticklabels = (label_names if col == 0 else 'auto'), cbar=False)

        # Add axis labels.
        if row + 1 == totl_rows:
            axs[row][col].set_xlabel('Predicted Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='x', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_xticks([])
            
        if col == 0:
            axs[row][col].set_ylabel('True Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='y', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_yticks([])
        
        axs[row][col].set_title("TREE DEPTH: " + str(lvl) + '    TEST ACC: ' + str(round(test_accuracy*100,2)) + '%', 
                                weight='bold', color = 'mediumblue', pad=14, loc='center', size=12)
        
        tree_df.loc['Experiment_' + str(i + 1), 'MAX DEPTH'] = lvl
        tree_df.loc['Experiment_' + str(i + 1), 'CV ACCURACY'] = scores.mean()
        tree_df.loc['Experiment_' + str(i + 1), 'TRAIN ACCURACY'] = accuracy.mean()
        tree_df.loc['Experiment_' + str(i + 1), 'TEST ACCURACY'] = test_accuracy
        
        i += 1
    plt.show()
    
    ## Plot mean CV accuracies for Tree Depth
    ax = plt.axes()
    plt.plot(t_depth, tree_df['TRAIN ACCURACY'], label = "Training Accuracy", 
         color = 'royalblue', marker = '.', lw = 1)
    plt.plot(t_depth, tree_df['CV ACCURACY'], label = "Cross-Val Accuracy", 
             color = 'mediumblue', marker = 'D', lw = 2)
    
    # Spine formatting
    ax.set_facecolor(color='white')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_color('cornflowerblue')
    ax.spines['bottom'].set_color('cornflowerblue')
    
    # Tick formatting
    ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
    ax.tick_params(axis='x', which='both', colors = 'gray')
    ax.set_xticks(t_depth)
    
    # Grid formatting
    ax.grid(False)
    vals = ax.get_yticks()
    for tick in vals:
        ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)
    
    # Set title
    plt.suptitle("Mean Accuracies vs. Decision Tree Depth", x = 0.123, y = 1.06, 
                 ha = 'left', weight='bold', color = 'mediumblue', 
                 size = 18)

    # Set title
    ax.set_title("Training and Cross-Validation", weight='normal', color = 'dimgray', 
                 style = 'italic', pad=14, loc='left', size=15)
    
    ax.legend(loc = 4, fontsize = 'medium', 
              shadow = True,
              edgecolor = 'blue',
              labelcolor = ['royalblue','mediumblue'], 
              facecolor = 'ivory')
    
    ax.set_xlabel('Tree Depth', color = 'gray', weight = 'bold')
    ax.set_ylabel('Mean Accuracy', color = 'gray', weight = 'bold')
    ax.set_ylim(top = 1.04)
    plt.show()
    
    return tree_df

## Plot accuracies vs. k values
experiments_tree_df = dt_acc_plot(1, 21, X_train_pca, Y_train, X_test_pca, Y_test)

experiments_tree_df

In [ ]:
# UNCOMMENT THIS TO GET A PICTURE OF THE TREE

dtree = tree.DecisionTreeClassifier(random_state = 0, max_depth = 5)
dtree.fit(X_train_pca, Y_train)

tree_predictions = dtree.predict(X_test_pca)

print('TestAccuracy: %1.4f' %dtree.score(X_test_pca, Y_test))

tree.plot_tree(dtree)

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Random Forest</div>

In [ ]:
# Create helper function for plotting the training accuracy vs. 
## cross-validation accuracy plot with various depths
def rf_acc_plot(start: int, 
                end: int, 
                x_train, 
                y_train,
                x_test,
                y_test):
    ## Set depth range and initialize lists
    rf_depth = range(start, end)
    tree_scores = []
    train_acc = []
    test_acc = []
    exp_ind = ['Experiment_' + str(i) for i in range(1,end-start+1)]
    rfor_df = pd.DataFrame(index = exp_ind)
    totl_cols = 4
    totl_rows = math.ceil((end-start-1)/totl_cols)
    
    fig, axs = plt.subplots(nrows = totl_rows, ncols= totl_cols, figsize = (5 * totl_cols, 4 * totl_rows))
    i = 0
    
    for lvl in rf_depth:
        row = math.trunc(i/totl_cols)
        col = i%totl_cols
        rfor = RandomForestClassifier(random_state = 0, max_depth = lvl)
        rfor.fit(x_train, y_train)
        accuracy = rfor.score(x_train, y_train)
        scores = cross_val_score(rfor, x_train, y_train, cv = 5, scoring='accuracy')
        test_accuracy = rfor.score(x_test, y_test)
        forest_predictions = rfor.predict(x_test)
        forest_confusion_matrix = tf.math.confusion_matrix(y_test, forest_predictions)
        
        sns.heatmap(ax = axs[row][col], data = forest_confusion_matrix, annot=True, fmt='.3g', cmap='Blues',
                    xticklabels = (label_names if row + 1 == totl_rows else 'auto'), 
                    yticklabels = (label_names if col == 0 else 'auto'), cbar=False)

        # Add axis labels.
        if row + 1 == totl_rows:
            axs[row][col].set_xlabel('Predicted Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='x', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_xticks([])
            
        if col == 0:
            axs[row][col].set_ylabel('True Label', color = 'gray', weight = 'bold')
            axs[row][col].tick_params(axis='y', which='both', length=0, colors = 'gray')
        else:
            axs[row][col].set_yticks([])
        
        axs[row][col].set_title("FOREST DEPTH: " + str(lvl) + '    TEST ACC: ' + str(round(test_accuracy*100,2)) + '%', 
                                weight='bold', color = 'mediumblue', pad=14, loc='center', size=12)
        
        rfor_df.loc['Experiment_' + str(i + 1), 'MAX DEPTH'] = lvl
        rfor_df.loc['Experiment_' + str(i + 1), 'CV ACCURACY'] = scores.mean()
        rfor_df.loc['Experiment_' + str(i + 1), 'TRAIN ACCURACY'] = accuracy.mean()
        rfor_df.loc['Experiment_' + str(i + 1), 'TEST ACCURACY'] = test_accuracy
        
        i += 1
    plt.show()
    
    ## Plot mean CV accuracies for depth
    
    ax = plt.axes()
    plt.plot(rf_depth, rfor_df['TRAIN ACCURACY'], label = "Training Accuracy", 
         color = 'royalblue', marker = '.', lw = 1)
    plt.plot(rf_depth, rfor_df['CV ACCURACY'], label = "Cross-Val Accuracy", 
             color = 'mediumblue', marker = 'D', lw = 2)
    
    # Spine formatting
    ax.set_facecolor(color='white')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_color('cornflowerblue')
    ax.spines['bottom'].set_color('cornflowerblue')
    
    # Tick formatting
    ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
    ax.tick_params(axis='x', which='both', colors = 'gray')
    ax.set_xticks(rf_depth)
    
    # Grid formatting
    ax.grid(False)
    vals = ax.get_yticks()
    for tick in vals:
        ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)
    
    # Set title
    plt.suptitle("Mean Accuracies vs. Random Forest Depth", x = 0.123, y = 1.06, 
                 ha = 'left', weight='bold', color = 'mediumblue', 
                 size = 18)

    # Set title
    ax.set_title("Training and Cross-Validation", weight='normal', color = 'dimgray', 
                 style = 'italic', pad=14, loc='left', size=15)
    
    ax.legend(loc = 4, fontsize = 'medium', 
              shadow = True,
              edgecolor = 'blue',
              labelcolor = ['royalblue','mediumblue'], 
              facecolor = 'ivory')
    
    ax.set_xlabel('Forest Depth', color = 'gray', weight = 'bold')
    ax.set_ylabel('Mean Accuracy', color = 'gray', weight = 'bold')
    ax.set_ylim(top = 1.04)
    plt.show()
    
    return rfor_df

## Plot accuracies vs. k values
experiments_rfor_df = rf_acc_plot(1, 13, X_train_pca, Y_train, X_test_pca, Y_test)

experiments_rfor_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Neural Networks </div>

In [ ]:
#Neural network with hidden layers

def build_nn_model(n_classes,
                   hidden_layer_sizes=[],
                   activation='relu',
                   optimizer='SGD',
                   learning_rate=0.01):
    """Build a multi-class logistic regression model using Keras.

    Args:
        n_classes: Number of output classes in the dataset.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer to use (SGD, Adam).
        learning_rate: The desired learning rate for the optimizer.

    Returns:
        model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(5678)
    tf.random.set_seed(5678)
    random.seed(5678)

    model = tf.keras.Sequential()

    # flatten the 128x128x3 images into 1-D vectors
    model.add(keras.layers.Flatten())
  
    # add the specified hidden layers with the specified activation
    i = 1
    for hidden_layer_size in hidden_layer_sizes:
        lay_name = 'Hidden_' + str(i)
        model.add(tf.keras.layers.Dense(units = hidden_layer_size,
                                        activation = activation,
                                        name = lay_name))
        print('Hidden Layer ' + str(i) + ' size: ' + str(hidden_layer_size))
        i += 1
  
    #Output layer. Since we are doing multi-class classification, use softmax for the activation
    model.add(tf.keras.layers.Dense(units = n_classes,
                                    activation = 'softmax',
                                    name = 'Output'))
    
    print('Activation function: ' + activation)

    if optimizer == 'SGD':
        model_opt = tf.keras.optimizers.SGD(learning_rate = learning_rate)
        print('Optimizer: SGD')
    elif optimizer == 'Adam':
        model_opt = keras.optimizers.Adam(learning_rate = learning_rate)
        print('Optimizer: Adam')
    else:
        model_opt = optimizer
        print('Optimizer: ' + optimizer)
        
    print('Learning Rate: ' + str(learning_rate))

    model.compile(loss='sparse_categorical_crossentropy', optimizer = model_opt, metrics=['accuracy'])

    return model


In [ ]:
def train_and_evaluate_nn(x_train,
                          y_train,
                          x_test,
                          y_test,
                          hidden_layer_sizes=[],
                          activation='relu',
                          optimizer='Adam',
                          learning_rate=0.01,
                          num_epochs=20, 
                          verb = False):

    # Build the model.
    model = build_nn_model(n_classes=4,
                           hidden_layer_sizes=hidden_layer_sizes,
                           activation=activation,
                           optimizer=optimizer,
                           learning_rate=learning_rate)

    # Train the model.
    history = model.fit(
        x = x_train,
        y = y_train,
        epochs = num_epochs,
        batch_size = 64, 
        verbose = verb,
        validation_split = 0.1)

    model.summary()

    # Retrieve the training metrics (after each train epoch) and the final test
    # accuracy.
    hist = history.history
    train_accuracy = hist['accuracy']
    val_accuracy = hist['val_accuracy']
    
    test_accuracy = model.evaluate(x = x_test, y = y_test, verbose = verb,
                                   return_dict=True)['accuracy']

    num_params = model.count_params()

    test_predictions = np.argmax(model.predict(x_test, verbose = verb), axis=-1)
  
    # Create a confusion matrix as a 2D array.
    confusion_matrix = tf.math.confusion_matrix(y_test, test_predictions)
    
    ## Plot mean CV accuracies for Tree Depth
    fig = plt.figure(figsize=(15, 5))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(train_accuracy, label = "Training Accuracy", 
         color = 'royalblue', marker = '.', lw = 1)
    ax.plot(val_accuracy, label = "Validation Accuracy", 
             color = 'mediumblue', marker = 'D', lw = 2)
    
    # Spine formatting
    ax.set_facecolor(color='white')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_color('cornflowerblue')
    ax.spines['bottom'].set_color('cornflowerblue')
    
    # Tick formatting
    ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
    ax.tick_params(axis='x', which='both', colors = 'gray')
    ax.set_xticks(range(num_epochs + 1))
    
    # Grid formatting
    ax.grid(False)
    vals = ax.get_yticks()
    for tick in vals:
        ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)
    
    # Set title
    plt.suptitle("Accuracy vs. Number of Epochs", x = 0.123, y = 1, 
                 ha = 'left', weight='bold', color = 'mediumblue', 
                 size = 18)

    # Set title
    ax.set_title("Training and Validation", weight='normal', color = 'dimgray', 
                 style = 'italic', pad=10, loc='left', size=15)
    
    ax.legend(loc = 4, fontsize = 'medium', 
              shadow = True,
              edgecolor = 'blue',
              labelcolor = ['royalblue','mediumblue'], 
              facecolor = 'ivory')
    
    ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
    ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
    ax.set_ylim(top = 1.01)

    # Use a heatmap plot to display it.
    ax = fig.add_subplot(1, 2, 2)
    ax = sns.heatmap(confusion_matrix, annot = True, fmt = '.3g', cmap = 'Blues',
                     xticklabels = label_names, yticklabels = label_names, cbar = False)
    
    # Add axis labels.
    ax.tick_params(axis='x', which='both', length=0, colors = 'gray', rotation = 90)
    ax.set_xlabel('Predicted Label', color = 'gray', weight = 'bold')
    ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
    ax.set_ylabel('True Label', color = 'gray', weight = 'bold')

    ax.set_title('CONFUSION MATRIX', 
                 weight='bold', color = 'mediumblue', pad=14, loc='left', size=18)

    ax.text(x = 4, y = -0.22, ha = 'right',
            s = 'TEST ACC: ' + str(round(test_accuracy*100,2)) + '%',
            color = 'mediumblue', weight='bold')
    
    plt.subplots_adjust(wspace=0.5)
    plt.show()
    
    metrics = compute_model_performance_metrics(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')

    return [test_accuracy, num_params, metrics]


In [ ]:
# Generating a Dataframe with all the Neural Network experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
acts = (['relu'] + ['tanh']* 2 + ['relu'] * 3)
opts = (['Adam'] + ['SGD'] * 3 + ['Adam'] * 2)
sizes = [[]] + [[128]] + [[256 , 128]] + ([[128]]) * 2 + [[256 , 128]]
rates = [0.01] * 3 + [0.0001] + [0.001] + [0.0001]
epchs = [21] + [5] + [20] * 4
params = [0] * 6
accs = [0] * 6

experiments_nndf = pd.DataFrame({'ACTIVATION': acts, 
                                 'OPTIMIZER': opts, 
                                 'HIDDEN SIZES': sizes, 
                                 'LEARNING RATE': rates,
                                 'EPOCHS': epchs, 
                                 '# PARAMETERS': params,
                                 'TEST ACCURACY': accs, 
                                 '# PARAMETERS (PCA)': params,
                                 'TEST ACCURACY (PCA)': accs}, 
                                index = exp_ind)

In [ ]:
for i in range (1,len(experiments_nndf.index)+1):
    print('Experiment_' + str(i) + '\n')
    accu, param, metrics = train_and_evaluate_nn(X_train, Y_train, X_test, Y_test,
                                        hidden_layer_sizes = experiments_nndf.loc['Experiment_' + str(i), 'HIDDEN SIZES'], 
                                        activation = experiments_nndf.loc['Experiment_' + str(i), 'ACTIVATION'], 
                                        optimizer = experiments_nndf.loc['Experiment_' + str(i), 'OPTIMIZER'], 
                                        learning_rate = experiments_nndf.loc['Experiment_' + str(i), 'LEARNING RATE'], 
                                        num_epochs = experiments_nndf.loc['Experiment_' + str(i), 'EPOCHS'])
    
    experiments_nndf.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_nndf.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    
    
    print('Experiment_' + str(i) + '_PCA\n')
    accu_pca, param_pca, metrics = train_and_evaluate_nn(X_train_pca, Y_train, X_test_pca, Y_test,
                                        hidden_layer_sizes = experiments_nndf.loc['Experiment_' + str(i), 'HIDDEN SIZES'], 
                                        activation = experiments_nndf.loc['Experiment_' + str(i), 'ACTIVATION'], 
                                        optimizer = experiments_nndf.loc['Experiment_' + str(i), 'OPTIMIZER'], 
                                        learning_rate = experiments_nndf.loc['Experiment_' + str(i), 'LEARNING RATE'], 
                                        num_epochs = experiments_nndf.loc['Experiment_' + str(i), 'EPOCHS'])
    
    experiments_nndf.loc['Experiment_' + str(i), '# PARAMETERS (PCA)'] = param_pca
    experiments_nndf.loc['Experiment_' + str(i), 'TEST ACCURACY (PCA)'] = accu_pca
    experiments_nndf.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_nndf.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    experiments_nndf.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_nndf.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_nndf.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]

In [ ]:
experiments_nndf

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">CNN</div>

In [ ]:
X_train_pca_sq = X_train_pca.reshape(X_train_pca.shape[0], int(np.sqrt(X_train_pca.shape[1])), int(np.sqrt(X_train_pca.shape[1])), 1)
X_test_pca_sq = X_test_pca.reshape(X_test_pca.shape[0], int(np.sqrt(X_test_pca.shape[1])), int(np.sqrt(X_test_pca.shape[1])), 1)

In [ ]:
def build_cnn_model(input_shape,
                    kern_sz = (5,5), 
                    stri_sz = (1,1), 
                    pool_sz = (2,2),
                    lear_rt = 0.001,
                    optimiz = 'Adam'):
    
    tf.keras.backend.clear_session()
    np.random.seed(5678)
    tf.random.set_seed(5678)
    random.seed(5678)
    model = tf.keras.Sequential()

    # add first convolution layer to the model
    model.add(tf.keras.layers.Conv2D(
        filters = 32,
        kernel_size = kern_sz,
        strides = stri_sz,
        padding = 'same',
        data_format = 'channels_last',
        name = 'conv_1',
        activation = 'relu'))


    # add a max pooling layer with pool size (2,2) and strides of 2
    # (this will reduce the spatial dimensions by half)
    model.add(tf.keras.layers.MaxPool2D(
        pool_size = pool_sz,
        name = 'pool_1'))


    # add second convolutional layer
    model.add(tf.keras.layers.Conv2D(
        filters = 64,
        kernel_size = kern_sz,
        strides = stri_sz,
        padding = 'same',
        name = 'conv_2',
        activation = 'relu'))

    # add second max pooling layer with pool size (2,2) and strides of 2
    # (this will further reduce the spatial dimensions by half)
    model.add(tf.keras.layers.MaxPool2D(
        pool_size = pool_sz, 
        name='pool_2'))


    # add a fully connected layer (need to flatten the output of the previous layers first)
    model.add(tf.keras.layers.Flatten()) 
    model.add(tf.keras.layers.Dense(
        units = 1024,
        name = 'fc_1', 
        activation = 'relu'))

    # add dropout layer
    model.add(tf.keras.layers.Dropout(
        rate = 0.5))

    # add the last fully connected layer
    # this last layer sets the activation function to "None" in order to output the logits 
    # note that passing activation = "sigmoid" will return class memembership probabilities but
    # in TensorFlow logits are prefered for numerical stability
    # set units=1 to get a single output unit (remember it's a binary classification problem)
    model.add(tf.keras.layers.Dense(
        units = 4,
        name = 'fc_2',
        activation = None))


    # build model and print summary
    model.build(input_shape = input_shape)
    
    if optimiz == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate = lear_rt)
    elif optimiz == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate = lear_rt)
    else:
        opt = optimiz
        print('Optimizer: ' + optimizer)
        
    model.compile(optimizer = opt,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
                  metrics = ['accuracy'])
    
    return model

In [ ]:
def build_n_run_cnn_model(x_train,
                          y_train,
                          x_test,
                          y_test,
                          kern_sz = (5,5), 
                          stri_sz = (1,1), 
                          pool_sz = (2,2),
                          lear_rt = 0.001,
                          optimiz = 'Adam', 
                          num_epochs = 20,
                          verb=False, 
                          graphing = True):

    model = build_cnn_model(x_train.shape, kern_sz, stri_sz, pool_sz, lear_rt, optimiz) 

    model.summary()
    
    history = model.fit(x_train, 
                        y_train,
                        epochs = num_epochs, 
                        validation_split=0.1,
                        verbose = verb)
    
    hist = history.history
    if (graphing):
        x_arr = np.arange(len(hist['loss'])) + 1
        
        ## Plot mean CV accuracies for Tree Depth
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(x_arr, hist['loss'], '-o', label='Train loss', 
             color = 'salmon', lw = 1)
        ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss', 
                 color = 'firebrick', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Loss vs. Number of Epochs", weight='bold', color = 'firebrick', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 1, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['salmon','firebrick'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Loss', color = 'gray', weight = 'bold')
        ax.set_ylim(bottom = -0.01)
        
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(x_arr, hist['accuracy'], '-o', label = "Training Accuracy", 
             color = 'royalblue', lw = 1)
        ax.plot(x_arr, hist['val_accuracy'], '--<', label = "Validation Accuracy", 
                 color = 'mediumblue', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Accuracy vs. Number of Epochs", weight='bold', color = 'mediumblue', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 4, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['royalblue','mediumblue'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
        ax.set_ylim(top = 1.01)
        
        plt.subplots_adjust(wspace=0.3)
        plt.show()

    test_accuracy = model.evaluate(x_test, y_test, verbose=0, return_dict=True)['accuracy']
    
    metrics = compute_model_performance_metrics(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')
    
    num_params = model.count_params()
    
    return [test_accuracy, num_params, metrics]

In [ ]:
# Generating a Dataframe with all the experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
kern_sz = [(5,5)] + [(3,3)] + [(5,5)] * 4
strides = [(1,1)] * 2 + [(2,2)] + [(1,1)] * 3
pool_sz = [(2,2)] * 3 + [(3,3)] + [(2,2)] * 2
lear_rt = [0.001] * 4 + [0.01] + [0.001]
optimiz = ['Adam'] * 5 + ['SGD']

experiments_cnn_df = pd.DataFrame({'kernel size': kern_sz, 
                               'strides': strides, 
                               'pool size': pool_sz, 
                               'learning rate': lear_rt, 
                               'optimizer': optimiz},
                             index = exp_ind)

In [ ]:
experiments_cnn_df 

In [ ]:
for i in range (1,len(experiments_cnn_df.index)+1):
    print('Experiment_' + str(i) + '\n')
    print(experiments_cnn_df.loc['Experiment_'+ str(i),'kernel size':'optimizer'], '\n')
    
    accu , param, metrics = build_n_run_cnn_model(X_train_pca_sq, Y_train, X_test_pca_sq, Y_test,
                                 kern_sz = experiments_cnn_df.loc['Experiment_' + str(i), 'kernel size'], 
                                 stri_sz = experiments_cnn_df.loc['Experiment_' + str(i), 'strides'],
                                 pool_sz = experiments_cnn_df.loc['Experiment_' + str(i), 'pool size'],
                                 lear_rt = experiments_cnn_df.loc['Experiment_' + str(i), 'learning rate'],
                                 optimiz = experiments_cnn_df.loc['Experiment_' + str(i), 'optimizer'])

    experiments_cnn_df.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_cnn_df.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    experiments_cnn_df.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_cnn_df.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    #experiments_cnn_df.loc['Experiment_' + str(i), 'ACCURACY'] = metrics["accuracy"]
    experiments_cnn_df.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_cnn_df.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_cnn_df.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]
    
    print('Test Accuracy: %1.4f' % accu + '\n\n')

In [ ]:
experiments_cnn_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;"> CNN + LSTM</div>

In [ ]:
print('Shape before X_train_pca_sq', X_train_pca_sq.shape)
print('Shape Y_train', Y_train.shape)

#reshape to time-series sequence of size 10
X_train_seq = np.reshape(X_train_pca_sq, (int(X_train_pca_sq.shape[0]/10), 10, X_train_pca_sq.shape[1], X_train_pca_sq.shape[2], X_train_pca_sq.shape[3]))
Y_train_seq = np.reshape(Y_train, (int(Y_train.shape[0]/10), 10))

X_test_seq = np.reshape(X_test_pca_sq, (int(X_test_pca_sq.shape[0]/10), 10, X_test_pca_sq.shape[1], X_test_pca_sq.shape[2], X_test_pca_sq.shape[3]))
Y_test_seq = np.reshape(Y_test, (int(Y_test.shape[0]/10), 10))

print('Shape X_train_seq', X_train_seq.shape)
print('Shape Y_train_seq', Y_train_seq.shape)

print('Shape X_test_seq', X_test_seq.shape)
print('Shape Y_test_seq', Y_test_seq.shape)

In [ ]:
def build_cnnlstm_model(input_shape,
                         kern_sz = (5,5), 
                         stri_sz = (1,1), 
                         pool_sz = (2,2),
                         lear_rt = 0.001,
                         optimiz = 'Adam'):
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()

    # add first convolution layer to the model
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(
        filters = 32,
        kernel_size = kern_sz,
        strides = stri_sz,
        padding = 'same',
        data_format = 'channels_last',
        name = 'conv_1',
        activation = 'relu')))

    # add a max pooling layer with pool size (2,2) and strides of 2
    # (this will reduce the spatial dimensions by half)
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D(
        pool_size = pool_sz,
        name = 'pool_1')))


    # add second convolutional layer
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(
        filters = 64,
        kernel_size = kern_sz,
        strides = stri_sz,
        padding = 'same',
        name = 'conv_2',
        activation = 'relu')))

    # add second max pooling layer with pool size (2,2) and strides of 2
    # (this will further reduce the spatial dimensions by half)
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D(
        pool_size = pool_sz, 
        name='pool_2')))


    # add a fully connected layer (need to flatten the output of the previous layers first)
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(
        units = 1024,
        name = 'fc_1', 
        activation = 'relu')))

    # add dropout layer
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(rate = 0.5)))

    #Add LSTM layers
    model.add(tf.keras.layers.LSTM(1024, return_sequences=True))
    model.add(tf.keras.layers.Dense(units=4, name='fc_2', activation=None))

    model.build(input_shape=input_shape)
    
    if optimiz == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate = lear_rt)
    elif optimiz == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate = lear_rt)
    else:
        opt = optimiz
        print('Optimizer: ' + optimizer)
    
    model.compile(optimizer=opt, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
              metrics=['accuracy'])
    
    return model

In [ ]:
def buildrun_cnnlstm_model(x_train,
                          y_train,
                          x_test,
                          y_test,
                          kern_sz = (5,5), 
                          stri_sz = (1,1), 
                          pool_sz = (2,2),
                          lear_rt = 0.001,
                          optimiz = 'Adam', 
                          num_epochs = 20,
                          verb=False, 
                          graphing = True):

    model = build_cnnlstm_model(x_train.shape, kern_sz, stri_sz, pool_sz, lear_rt, optimiz) 

    model.summary()
    
    history = model.fit(x_train, 
                        y_train,
                        epochs = num_epochs, 
                        validation_split=0.1,
                        verbose = verb)
    
    hist = history.history
    if (graphing):
        x_arr = np.arange(len(hist['loss'])) + 1
        
        ## Plot mean CV accuracies for Tree Depth
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(x_arr, hist['loss'], '-o', label='Train loss', 
             color = 'salmon', lw = 1)
        ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss', 
                 color = 'firebrick', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Loss vs. Number of Epochs", weight='bold', color = 'firebrick', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 1, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['salmon','firebrick'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Loss', color = 'gray', weight = 'bold')
        ax.set_ylim(bottom = -0.01)
        
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(x_arr, hist['accuracy'], '-o', label = "Training Accuracy", 
             color = 'royalblue', lw = 1)
        ax.plot(x_arr, hist['val_accuracy'], '--<', label = "Validation Accuracy", 
                 color = 'mediumblue', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Accuracy vs. Number of Epochs", weight='bold', color = 'mediumblue', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 4, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['royalblue','mediumblue'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
        ax.set_ylim(top = 1.01)
        
        plt.subplots_adjust(wspace=0.3)
        plt.show()

    test_accuracy = model.evaluate(x_test, y_test, verbose=0, return_dict=True)['accuracy']
    
    metrics = compute_model_performance_metrics_cnn_lstm(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')
    
    num_params = model.count_params()
    
    return [test_accuracy, num_params, metrics]

In [ ]:
# Generating a Dataframe with all the experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
kern_sz = [(5,5)] + [(3,3)] + [(5,5)] * 4
strides = [(1,1)] * 2 + [(2,2)] + [(1,1)] * 3
pool_sz = [(2,2)] * 3 + [(3,3)] + [(2,2)] * 2
lear_rt = [0.001] * 4 + [0.01] + [0.001]
optimiz = ['Adam'] * 5 + ['SGD']

experiments_cnnlstm_df = pd.DataFrame({'kernel size': kern_sz, 
                                       'strides': strides, 
                                       'pool size': pool_sz, 
                                       'learning rate': lear_rt, 
                                       'optimizer': optimiz},
                                     index = exp_ind)

In [ ]:
for i in range (1,len(experiments_cnnlstm_df.index)+1):
    print('Experiment_' + str(i) + '\n')
    print(experiments_cnnlstm_df.loc['Experiment_'+ str(i),'kernel size':'optimizer'], '\n')
    
    accu , param, metrics = buildrun_cnnlstm_model(X_train_seq, Y_train_seq, X_test_seq, Y_test_seq,
                                 kern_sz = experiments_cnnlstm_df.loc['Experiment_' + str(i), 'kernel size'], 
                                 stri_sz = experiments_cnnlstm_df.loc['Experiment_' + str(i), 'strides'],
                                 pool_sz = experiments_cnnlstm_df.loc['Experiment_' + str(i), 'pool size'],
                                 lear_rt = experiments_cnnlstm_df.loc['Experiment_' + str(i), 'learning rate'],
                                 optimiz = experiments_cnnlstm_df.loc['Experiment_' + str(i), 'optimizer'])

    experiments_cnnlstm_df.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_cnnlstm_df.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    experiments_cnnlstm_df.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_cnnlstm_df.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    #experiments_cnnlstm_df.loc['Experiment_' + str(i), 'ACCURACY'] = metrics["accuracy"]
    experiments_cnnlstm_df.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_cnnlstm_df.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_cnnlstm_df.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]
    
    print('Test Accuracy: %1.4f' % accu + '\n\n')

In [ ]:
experiments_cnnlstm_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">RNN</div>

In [ ]:
def build_rnn_model(n_classes,
                    activation= 'relu',
                    optimizer= 'Adam',
                    learning_rate=0.01):
    
    print('Activation function: ' + activation)
    print('Optimizer: ' + optimizer)
    print('Learning Rate: ' + str(learning_rate))
    
    tf.keras.backend.clear_session()

    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(None,15)))

    model.add(
        tf.keras.layers.SimpleRNN(225, return_sequences = True, activation = activation)
    )


    model.add(
        tf.keras.layers.SimpleRNN(225, activation = activation)
    )

    model.add(tf.keras.layers.Dense(n_classes))

    if optimizer == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    else:
        opt = optimiz
        print('Optimizer: ' + optimizer)

    model.compile(optimizer = opt,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
                  metrics = ['accuracy'])

    return model

In [ ]:
def build_n_run_rnn_model(x_train,
                          y_train,
                          x_test,
                          y_test,
                          activ= 'relu',
                          lear_rt = 0.001,
                          optimiz = 'Adam', 
                          num_epochs = 10,
                          verb = False, 
                          graphing = True):
    
    print('Number of Epochs: ' + str(num_epochs))
    
    model = build_rnn_model(len(label_names), activ, optimiz, lear_rt) 
    history = model.fit(x_train, 
                        y_train,
                        batch_size = 64,
                        epochs = num_epochs, 
                        validation_split=0.1,
                        verbose = verb)
    
    hist = history.history
    if (graphing):
        x_arr = np.arange(len(hist['loss'])) + 1
        
        ## Plot mean CV accuracies for Tree Depth
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(x_arr, hist['loss'], '-o', label='Train loss', 
             color = 'salmon', lw = 1)
        ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss', 
                 color = 'firebrick', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Loss vs. Number of Epochs", weight='bold', color = 'firebrick', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 1, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['salmon','firebrick'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Loss', color = 'gray', weight = 'bold')
        ax.set_ylim(bottom = -0.01)
        
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(x_arr, hist['accuracy'], '-o', label = "Training Accuracy", 
             color = 'royalblue', lw = 1)
        ax.plot(x_arr, hist['val_accuracy'], '--<', label = "Validation Accuracy", 
                 color = 'mediumblue', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Accuracy vs. Number of Epochs", weight='bold', color = 'mediumblue', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 4, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['royalblue','mediumblue'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
        ax.set_ylim(top = 1.01)
        
        plt.subplots_adjust(wspace=0.3)
        plt.show()

    test_accuracy = model.evaluate(x_test, y_test, verbose=verb, return_dict=True)['accuracy']
    
    metrics = compute_model_performance_metrics(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')
    
    num_params = model.count_params()
    
    return [test_accuracy, num_params, metrics]

In [ ]:
# Generating a Dataframe with all the experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
activ = ['relu'] * 2 + ['tanh'] * 2 +  ['relu'] * 2
lear_rt = ([0.01] + [0.001]) * 3
optimiz = ['Adam'] *4 + ['SGD'] * 2

experiments_rnn_df = pd.DataFrame({'activation': activ, 
                                   'learning rate': lear_rt, 
                                   'optimizer': optimiz},
                             index = exp_ind)

In [ ]:
for i in range (1,len(experiments_rnn_df.index)+1):
    print('Experiment_' + str(i) + '\n')
    
    accu , param, metrics = build_n_run_rnn_model(X_train_pca_sq, Y_train, X_test_pca_sq, Y_test,
                                                  activ= experiments_rnn_df.loc['Experiment_' + str(i), 'activation'],
                                                  lear_rt = experiments_rnn_df.loc['Experiment_' + str(i), 'learning rate'],
                                                  optimiz = experiments_rnn_df.loc['Experiment_' + str(i), 'optimizer'])

    experiments_rnn_df.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_rnn_df.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    experiments_rnn_df.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_rnn_df.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    #experiments_cnn_df.loc['Experiment_' + str(i), 'ACCURACY'] = metrics["accuracy"]
    experiments_rnn_df.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_rnn_df.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_rnn_df.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]
    
    print('Test Accuracy: %1.4f' % accu + '\n\n')

In [ ]:
experiments_rnn_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">GRU</div>

In [ ]:
def build_gru_model(n_classes,
                    activation= 'relu',
                    optimizer= 'Adam',
                    learning_rate=0.01):
    
    print('Activation function: ' + activation)
    print('Optimizer: ' + optimizer)
    print('Learning Rate: ' + str(learning_rate))
    
    tf.keras.backend.clear_session()

    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(None,15)))

    model.add(
        tf.keras.layers.GRU(225, return_sequences = True, activation = activation)
    )


    model.add(
        tf.keras.layers.GRU(225, activation = activation)
    )

    model.add(tf.keras.layers.Dense(n_classes))

    if optimizer == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    else:
        opt = optimiz
        print('Optimizer: ' + optimizer)

    model.compile(optimizer = opt,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
                  metrics = ['accuracy'])

    return model

In [ ]:
def build_n_run_gru_model(x_train,
                          y_train,
                          x_test,
                          y_test,
                          activ= 'relu',
                          lear_rt = 0.001,
                          optimiz = 'Adam', 
                          num_epochs = 10,
                          verb = False, 
                          graphing = True):
    
    print('Number of Epochs: ' + str(num_epochs))
    
    model = build_gru_model(len(label_names), activ, optimiz, lear_rt) 
    history = model.fit(x_train, 
                        y_train,
                        batch_size = 64,
                        epochs = num_epochs, 
                        validation_split=0.1,
                        verbose = verb)
    
    hist = history.history
    if (graphing):
        x_arr = np.arange(len(hist['loss'])) + 1
        
        ## Plot mean CV accuracies for Tree Depth
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(x_arr, hist['loss'], '-o', label='Train loss', 
             color = 'salmon', lw = 1)
        ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss', 
                 color = 'firebrick', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Loss vs. Number of Epochs", weight='bold', color = 'firebrick', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 1, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['salmon','firebrick'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Loss', color = 'gray', weight = 'bold')
        ax.set_ylim(bottom = -0.01)
        
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(x_arr, hist['accuracy'], '-o', label = "Training Accuracy", 
             color = 'royalblue', lw = 1)
        ax.plot(x_arr, hist['val_accuracy'], '--<', label = "Validation Accuracy", 
                 color = 'mediumblue', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Accuracy vs. Number of Epochs", weight='bold', color = 'mediumblue', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 4, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['royalblue','mediumblue'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
        ax.set_ylim(top = 1.01)
        
        plt.subplots_adjust(wspace=0.3)
        plt.show()

    test_accuracy = model.evaluate(x_test, y_test, verbose=verb, return_dict=True)['accuracy']
    
    metrics = compute_model_performance_metrics(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')
    
    num_params = model.count_params()
    
    return [test_accuracy, num_params, metrics]

In [ ]:
# Generating a Dataframe with all the experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
activ = ['relu'] * 2 + ['tanh'] * 2 +  ['relu'] * 2
lear_rt = ([0.01] + [0.001]) * 3
optimiz = ['Adam'] *4 + ['SGD'] * 2

experiments_gru_df = pd.DataFrame({'activation': activ, 
                                   'learning rate': lear_rt, 
                                   'optimizer': optimiz},
                             index = exp_ind)

In [ ]:
for i in range (1,len(experiments_gru_df.index)+1):
    print('Experiment_' + str(i) + '\n')
    
    accu , param, metrics = build_n_run_gru_model(X_train_pca_sq, Y_train, X_test_pca_sq, Y_test,
                                                  activ= experiments_gru_df.loc['Experiment_' + str(i), 'activation'],
                                                  lear_rt = experiments_gru_df.loc['Experiment_' + str(i), 'learning rate'],
                                                  optimiz = experiments_gru_df.loc['Experiment_' + str(i), 'optimizer'])

    experiments_gru_df.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_gru_df.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    experiments_gru_df.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_gru_df.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    #experiments_cnn_df.loc['Experiment_' + str(i), 'ACCURACY'] = metrics["accuracy"]
    experiments_gru_df.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_gru_df.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_gru_df.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]
    
    print('Test Accuracy: %1.4f' % accu + '\n\n')

In [ ]:
experiments_gru_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">RNN + GRU</div>

In [ ]:
def build_rnngru_model(n_classes,
                    activation= 'relu',
                    optimizer= 'Adam',
                    learning_rate=0.01):
    
    print('Activation function: ' + activation)
    print('Optimizer: ' + optimizer)
    print('Learning Rate: ' + str(learning_rate))
    
    tf.keras.backend.clear_session()

    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(None,15)))

    model.add(
        tf.keras.layers.SimpleRNN(225, return_sequences = True, activation = activation)
    )


    model.add(
        tf.keras.layers.GRU(225, activation = activation)
    )

    model.add(tf.keras.layers.Dense(n_classes))

    if optimizer == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    else:
        opt = optimiz
        print('Optimizer: ' + optimizer)

    model.compile(optimizer = opt,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
                  metrics = ['accuracy'])

    return model

In [ ]:
def build_n_run_rnngru_model(x_train,
                          y_train,
                          x_test,
                          y_test,
                          activ= 'relu',
                          lear_rt = 0.001,
                          optimiz = 'Adam', 
                          num_epochs = 10,
                          verb = False, 
                          graphing = True):
    
    print('Number of Epochs: ' + str(num_epochs))
    
    model = build_rnngru_model(len(label_names), activ, optimiz, lear_rt) 
    history = model.fit(x_train, 
                        y_train,
                        batch_size = 64,
                        epochs = num_epochs, 
                        validation_split=0.1,
                        verbose = verb)
    
    hist = history.history
    if (graphing):
        x_arr = np.arange(len(hist['loss'])) + 1
        
        ## Plot mean CV accuracies for Tree Depth
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(x_arr, hist['loss'], '-o', label='Train loss', 
             color = 'salmon', lw = 1)
        ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss', 
                 color = 'firebrick', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Loss vs. Number of Epochs", weight='bold', color = 'firebrick', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 1, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['salmon','firebrick'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Loss', color = 'gray', weight = 'bold')
        ax.set_ylim(bottom = -0.01)
        
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(x_arr, hist['accuracy'], '-o', label = "Training Accuracy", 
             color = 'royalblue', lw = 1)
        ax.plot(x_arr, hist['val_accuracy'], '--<', label = "Validation Accuracy", 
                 color = 'mediumblue', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Accuracy vs. Number of Epochs", weight='bold', color = 'mediumblue', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 4, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['royalblue','mediumblue'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
        ax.set_ylim(top = 1.01)
        
        plt.subplots_adjust(wspace=0.3)
        plt.show()

    test_accuracy = model.evaluate(x_test, y_test, verbose=verb, return_dict=True)['accuracy']
    
    metrics = compute_model_performance_metrics(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')
    
    num_params = model.count_params()
    
    return [test_accuracy, num_params, metrics]

In [ ]:
# Generating a Dataframe with all the experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
activ = ['relu'] * 2 + ['tanh'] * 2 +  ['relu'] * 2
lear_rt = ([0.01] + [0.001]) * 3
optimiz = ['Adam'] *4 + ['SGD'] * 2

experiments_rnngru_df = pd.DataFrame({'activation': activ, 
                                   'learning rate': lear_rt, 
                                   'optimizer': optimiz},
                             index = exp_ind)

In [ ]:
for i in range (1,len(experiments_rnngru_df.index)+1):
    print('Experiment_' + str(i) + '\n')
    
    accu , param, metrics = build_n_run_rnngru_model(X_train_pca_sq, Y_train, X_test_pca_sq, Y_test,
                                                  activ= experiments_rnngru_df.loc['Experiment_' + str(i), 'activation'],
                                                  lear_rt = experiments_rnngru_df.loc['Experiment_' + str(i), 'learning rate'],
                                                  optimiz = experiments_rnngru_df.loc['Experiment_' + str(i), 'optimizer'])

    experiments_rnngru_df.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_rnngru_df.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    experiments_rnngru_df.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_rnngru_df.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    #experiments_rnngru_df.loc['Experiment_' + str(i), 'ACCURACY'] = metrics["accuracy"]
    experiments_rnngru_df.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_rnngru_df.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_rnngru_df.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]
    
    print('Test Accuracy: %1.4f' % accu + '\n\n')

In [ ]:
experiments_rnngru_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;"> GRU + LSTM</div>

In [ ]:
def build_grulstm_model(n_classes,
                    activation= 'relu',
                    optimizer= 'Adam',
                    learning_rate=0.01):
    
    print('Activation function: ' + activation)
    print('Optimizer: ' + optimizer)
    print('Learning Rate: ' + str(learning_rate))
    
    tf.keras.backend.clear_session()

    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(None,15)))

    model.add(
        tf.keras.layers.GRU(225, return_sequences = True, activation = activation)
    )


    model.add(
        tf.keras.layers.LSTM(225, activation = activation)
    )

    model.add(tf.keras.layers.Dense(n_classes))

    if optimizer == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    else:
        opt = optimiz
        print('Optimizer: ' + optimizer)

    model.compile(optimizer = opt,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
                  metrics = ['accuracy'])

    return model

In [ ]:
def build_n_run_grulstm_model(x_train,
                          y_train,
                          x_test,
                          y_test,
                          activ= 'relu',
                          lear_rt = 0.001,
                          optimiz = 'Adam', 
                          num_epochs = 10,
                          verb = False, 
                          graphing = True):
    
    print('Number of Epochs: ' + str(num_epochs))
    
    model = build_grulstm_model(len(label_names), activ, optimiz, lear_rt) 
    history = model.fit(x_train, 
                        y_train,
                        batch_size = 64,
                        epochs = num_epochs, 
                        validation_split=0.1,
                        verbose = verb)
    
    hist = history.history
    if (graphing):
        x_arr = np.arange(len(hist['loss'])) + 1
        
        ## Plot mean CV accuracies for Tree Depth
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(x_arr, hist['loss'], '-o', label='Train loss', 
             color = 'salmon', lw = 1)
        ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss', 
                 color = 'firebrick', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Loss vs. Number of Epochs", weight='bold', color = 'firebrick', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 1, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['salmon','firebrick'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Loss', color = 'gray', weight = 'bold')
        ax.set_ylim(bottom = -0.01)
        
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(x_arr, hist['accuracy'], '-o', label = "Training Accuracy", 
             color = 'royalblue', lw = 1)
        ax.plot(x_arr, hist['val_accuracy'], '--<', label = "Validation Accuracy", 
                 color = 'mediumblue', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Accuracy vs. Number of Epochs", weight='bold', color = 'mediumblue', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 4, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['royalblue','mediumblue'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
        ax.set_ylim(top = 1.01)
        
        plt.subplots_adjust(wspace=0.3)
        plt.show()

    test_accuracy = model.evaluate(x_test, y_test, verbose=verb, return_dict=True)['accuracy']
    
    metrics = compute_model_performance_metrics(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')
    
    num_params = model.count_params()
    
    return [test_accuracy, num_params, metrics]

In [ ]:
# Generating a Dataframe with all the experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
activ = ['relu'] * 2 + ['tanh'] * 2 +  ['relu'] * 2
lear_rt = ([0.01] + [0.001]) * 3
optimiz = ['SGD'] * 2 + ['Adam'] *4

experiments_grulstm_df = pd.DataFrame({'activation': activ, 
                                   'learning rate': lear_rt, 
                                   'optimizer': optimiz},
                             index = exp_ind)

In [ ]:
for i in range (1,len(experiments_grulstm_df.index)+1):
    print('Experiment_' + str(i) + '\n')
    
    accu , param, metrics = build_n_run_grulstm_model(X_train_pca_sq, Y_train, X_test_pca_sq, Y_test,
                                                  activ= experiments_grulstm_df.loc['Experiment_' + str(i), 'activation'],
                                                  lear_rt = experiments_grulstm_df.loc['Experiment_' + str(i), 'learning rate'],
                                                  optimiz = experiments_grulstm_df.loc['Experiment_' + str(i), 'optimizer'])

    experiments_grulstm_df.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_grulstm_df.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    experiments_grulstm_df.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_grulstm_df.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    #experiments_grulstm_df.loc['Experiment_' + str(i), 'ACCURACY'] = metrics["accuracy"]
    experiments_grulstm_df.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_grulstm_df.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_grulstm_df.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]
    
    print('Test Accuracy: %1.4f' % accu + '\n\n')

In [ ]:
experiments_grulstm_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;"> Bidirectional LSTM + GRU</div>

In [ ]:
def build_blstmgru_model(n_classes,
                    activation= 'relu',
                    optimizer= 'Adam',
                    learning_rate=0.01):
    
    print('Activation function: ' + activation)
    print('Optimizer: ' + optimizer)
    print('Learning Rate: ' + str(learning_rate))
    
    tf.keras.backend.clear_session()

    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(None,15)))

    model.add(
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(225, return_sequences = True, activation = activation)
        )
    )


    model.add(
        tf.keras.layers.GRU(225, activation = activation)
    )

    model.add(tf.keras.layers.Dense(n_classes))

    if optimizer == 'SGD':
        opt = tf.keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    else:
        opt = optimiz
        print('Optimizer: ' + optimizer)

    model.compile(optimizer = opt,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
                  metrics = ['accuracy'])

    return model

In [ ]:
def build_n_run_blstmgru_model(x_train,
                          y_train,
                          x_test,
                          y_test,
                          activ= 'relu',
                          lear_rt = 0.001,
                          optimiz = 'Adam', 
                          num_epochs = 10,
                          verb = False, 
                          graphing = True):
    
    print('Number of Epochs: ' + str(num_epochs))
    
    model = build_blstmgru_model(len(label_names), activ, optimiz, lear_rt) 
    history = model.fit(x_train, 
                        y_train,
                        batch_size = 64,
                        epochs = num_epochs, 
                        validation_split=0.1,
                        verbose = verb)
    
    hist = history.history
    if (graphing):
        x_arr = np.arange(len(hist['loss'])) + 1
        
        ## Plot mean CV accuracies for Tree Depth
        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(x_arr, hist['loss'], '-o', label='Train loss', 
             color = 'salmon', lw = 1)
        ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss', 
                 color = 'firebrick', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Loss vs. Number of Epochs", weight='bold', color = 'firebrick', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 1, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['salmon','firebrick'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Loss', color = 'gray', weight = 'bold')
        ax.set_ylim(bottom = -0.01)
        
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(x_arr, hist['accuracy'], '-o', label = "Training Accuracy", 
             color = 'royalblue', lw = 1)
        ax.plot(x_arr, hist['val_accuracy'], '--<', label = "Validation Accuracy", 
                 color = 'mediumblue', lw = 2)

        # Spine formatting
        ax.set_facecolor(color='white')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('cornflowerblue')
        ax.spines['bottom'].set_color('cornflowerblue')

        # Tick formatting
        ax.tick_params(axis='y', which='both', length=0, colors = 'gray')
        ax.tick_params(axis='x', which='both', colors = 'gray')
        ax.set_xticks(range(num_epochs + 1))

        # Grid formatting
        ax.grid(False)
        vals = ax.get_yticks()
        for tick in vals:
            ax.axhline(y=tick, linestyle='dashed', alpha=0.4, color='gainsboro', zorder=1)

        # Set title
        ax.set_title("Accuracy vs. Number of Epochs", weight='bold', color = 'mediumblue', 
                     pad=10, loc='left', size = 18 )

        ax.legend(loc = 4, fontsize = 'medium', 
                  shadow = True,
                  edgecolor = 'blue',
                  labelcolor = ['royalblue','mediumblue'], 
                  facecolor = 'ivory')

        ax.set_xlabel('Train epochs', color = 'gray', weight = 'bold')
        ax.set_ylabel('Accuracy', color = 'gray', weight = 'bold')
        ax.set_ylim(top = 1.01)
        
        plt.subplots_adjust(wspace=0.3)
        plt.show()

    test_accuracy = model.evaluate(x_test, y_test, verbose=verb, return_dict=True)['accuracy']
    
    metrics = compute_model_performance_metrics(model, x_test, y_test)
    
    train_acc = float(hist['accuracy'][len(hist)-1])
    val_acc = float(hist['val_accuracy'][len(hist)-1])
    
    metrics["train_accuracy"] = format(train_acc, '.4f')
    metrics["val_accuracy"] = format(val_acc, '.4f')
    
    num_params = model.count_params()
    
    return [test_accuracy, num_params, metrics]

In [ ]:
# Generating a Dataframe with all the experiments' information

exp_ind = ['Experiment_' + str(i) for i in range(1,7)]
activ = ['relu'] * 2 + ['tanh'] * 2 +  ['relu'] * 2
lear_rt = ([0.01] + [0.001]) * 3
optimiz = ['SGD'] * 2 + ['Adam'] *4

experiments_blstmgru_df = pd.DataFrame({'activation': activ, 
                                   'learning rate': lear_rt, 
                                   'optimizer': optimiz},
                             index = exp_ind)

In [ ]:
for i in range (1,len(experiments_blstmgru_df.index)+1):
    print('Experiment_' + str(i) + '\n')
    
    accu , param, metrics = build_n_run_blstmgru_model(X_train_pca_sq, Y_train, X_test_pca_sq, Y_test,
                                                  activ= experiments_blstmgru_df.loc['Experiment_' + str(i), 'activation'],
                                                  lear_rt = experiments_blstmgru_df.loc['Experiment_' + str(i), 'learning rate'],
                                                  optimiz = experiments_blstmgru_df.loc['Experiment_' + str(i), 'optimizer'])

    experiments_blstmgru_df.loc['Experiment_' + str(i), '# PARAMETERS'] = param
    experiments_blstmgru_df.loc['Experiment_' + str(i), 'TEST ACCURACY'] = accu
    experiments_blstmgru_df.loc['Experiment_' + str(i), 'TRAIN ACCURACY'] = metrics["train_accuracy"]
    experiments_blstmgru_df.loc['Experiment_' + str(i), 'VAL ACCURACY'] = metrics["val_accuracy"]
    #experiments_blstmgru_df.loc['Experiment_' + str(i), 'ACCURACY'] = metrics["accuracy"]
    experiments_blstmgru_df.loc['Experiment_' + str(i), 'PRECISON'] = metrics["precision"]
    experiments_blstmgru_df.loc['Experiment_' + str(i), 'RECALL'] = metrics["recall"]
    experiments_blstmgru_df.loc['Experiment_' + str(i), 'F1 SCORE'] = metrics["f1_score"]
    
    print('Test Accuracy: %1.4f' % accu + '\n\n')

In [ ]:
experiments_blstmgru_df

<div style="padding:10px;background-color: cornflowerblue; color:white;font-size:28px;">Support Vector Machine (SVM) </div>

In [ ]:
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}

svc=svm.SVC(
        probability=True 
        #,verbose=True enable for debug
        )

model=RandomizedSearchCV(
        svc, 
        param_grid, 
        cv=3, 
        n_jobs=-1 # use all cores
        #,verbose=4 enable for debug
    )

In [ ]:
#Train model
model.fit(X_train_pca,Y_train)

In [ ]:
#print results

Y_pred = model.predict(X_test_pca)

accuracy = accuracy_score(Y_test,Y_pred)
precision = precision_score(Y_test, Y_pred, average='macro')
recall = recall_score(Y_test, Y_pred, average='macro')
f1 = f1_score(Y_test, Y_pred, average='macro')

print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('F1 score: ', f1)


In [ ]:
# model.best_params_ contains the best parameters obtained from GridSearchCV
model.best_params_

In [ ]:
end = timer()

In [ ]:
print(end - start)